## 어떤 필터링도 적용 안함

In [1]:
import os, sys
os.environ["SPARK_HOME"] = "/opt/homebrew/opt/apache-spark/libexec"
sys.path.insert(0, os.environ["SPARK_HOME"] + "/python")

In [2]:
!pyspark --version

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 4.1.1
      /_/
                        
Using Scala version 2.13.17, OpenJDK 64-Bit Server VM, 21.0.9
Branch HEAD
Compiled by user runner on 2026-01-02T11:55:02Z
Revision c0690c763bafabd08e7079d1137fa0a769a05bae
Url https://github.com/apache/spark
Type --help for more information.


In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("demo").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/26 10:13:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
sc = spark.sparkContext

In [7]:
stopwords = (
    spark.sparkContext
         .textFile("/Users/js/tasteam-aicode-gpu-all-python-process-runtime_for_github/hybrid_search/data_preprocessing/stopwords-ko.txt")
         .collect()
)

stopwords_set = set(stopwords)


In [8]:
bc_stopwords = spark.sparkContext.broadcast(stopwords_set)

In [9]:
import re

df = spark.read.option("sep", "\t").option("header", "true").csv("/Users/js/tasteam-aicode-gpu-all-python-process-runtime_for_github/data/kr3.tsv")

rdd = (
    df.select("Review")
      .rdd
      .flatMap(lambda r: re.findall(r"[가-힣A-Za-z0-9]+", r[0]))
      .filter(lambda w: w not in bc_stopwords.value)
      .filter(lambda w: len(w) >= 2)          # 한 글자 제거 (선택)
      .map(lambda w: (w, 1))
      .reduceByKey(lambda a, b: a + b)
)
rdd.collect()

[('돼지고기', 2839),
 ('쾌적한', 492),
 ('편이고', 3320),
 ('인상', 2425),
 ('알았는데', 3642),
 ('육향', 334),
 ('수육이', 657),
 ('소주를', 534),
 ('시킨', 4412),
 ('참고하세요', 1654),
 ('무난한', 3852),
 ('순댓국보다', 37),
 ('않고', 29799),
 ('친절했음', 124),
 ('먹음', 5539),
 ('고기에서', 392),
 ('별로', 16413),
 ('없는', 21428),
 ('순댓국집이죠', 2),
 ('편리합니다', 89),
 ('대기인원이', 68),
 ('주차는', 2796),
 ('푸짐하다', 446),
 ('찹쌀순대는', 10),
 ('않은', 15668),
 ('하얀', 1468),
 ('후추', 1327),
 ('조합해', 39),
 ('당면이', 254),
 ('동네에선', 98),
 ('지금이야', 32),
 ('힘들어졌다', 12),
 ('국물을', 2428),
 ('주문했는데', 5072),
 ('10분', 1793),
 ('싶을', 8129),
 ('미리', 5990),
 ('지금까지', 2017),
 ('저녁엔', 387),
 ('마파', 76),
 ('냄새', 2679),
 ('버섯', 3778),
 ('탕수육에는', 8),
 ('미흡했으나', 1),
 ('이미', 5488),
 ('하여', 1607),
 ('마파두부가', 113),
 ('맛있다고', 7543),
 ('올라와서', 336),
 ('길었음', 38),
 ('오래', 6765),
 ('흔들릴', 9),
 ('pm710', 1),
 ('주문한', 4121),
 ('숟가락을', 181),
 ('감동은', 318),
 ('기다려야', 2454),
 ('신선한', 4128),
 ('드디어', 3802),
 ('먹게', 3591),
 ('먹길래', 120),
 ('우리도', 155),
 ('식구', 44),
 ('티브이에서', 131),
 ('생각나

In [10]:
sorted_rdd = rdd.sortBy(lambda x: x[1], ascending=False)
sorted_rdd.take(30)

[('않고', 29799),
 ('먹었는데', 25643),
 ('고기', 25547),
 ('양이', 25030),
 ('먹으면', 22287),
 ('음식', 22004),
 ('분위기', 21628),
 ('가격', 21597),
 ('없는', 21428),
 ('커피', 21315),
 ('맛을', 20317),
 ('카페', 20143),
 ('메뉴', 19471),
 ('처음', 19444),
 ('살짝', 19399),
 ('가게', 19371),
 ('제일', 19202),
 ('매우', 18777),
 ('가서', 18627),
 ('특히', 18623),
 ('가장', 18262),
 ('맛있었다', 18192),
 ('저는', 17858),
 ('가격이', 17830),
 ('자주', 17599),
 ('있어요', 17528),
 ('생각보다', 17456),
 ('추천', 17297),
 ('많아서', 17127),
 ('양도', 16931)]

In [11]:
import re
from itertools import islice

# 1) 불용어 로드 + broadcast (명사 불용어 중심으로)
with open("stopwords-ko.txt", encoding="utf-8") as f:
    stopwords = set(w.strip() for w in f if w.strip())
bc_stop = spark.sparkContext.broadcast(stopwords)

# 2) 파티션 단위로 Kiwi 1번만 초기화해서 명사/NNG/NNP만 추출
def extract_noun_bigrams_partition(rows):
    from kiwipiepy import Kiwi
    kiwi = Kiwi()  # ✅ 파티션당 1번
    stop = bc_stop.value

    for row in rows:
        text = row[0]
        if not text:
            continue

        # (선택) 너무 긴/이상한 문자 정리
        text = re.sub(r"\s+", " ", text).strip()

        tokens = []
        for tok in kiwi.tokenize(text):
            # Kiwi 품사 예: NNG(일반명사), NNP(고유명사)
            if tok.tag in ("NNG", "NNP"):
                w = tok.form
                if len(w) >= 2 and w not in stop:
                    tokens.append(w)

        # 명사 bigram 생성
        for a, b in zip(tokens, tokens[1:]):
            yield (f"{a} {b}", 1)

# 3) 실행 파이프라인
bigram_counts = (
    df.select("Review")
      .rdd
      .mapPartitions(extract_noun_bigrams_partition)
      .reduceByKey(lambda a, b: a + b)
      .filter(lambda kv: kv[0].split()[0] != kv[0].split()[1])
)

top_bigrams = bigram_counts.takeOrdered(50, key=lambda x: -x[1])
top_bigrams[:10]

Quantization is not supported for ArchType::Quantization is not supported for ArchType::Quantization is not supported for ArchType::Quantization is not supported for ArchType::Quantization is not supported for ArchType::neonneon. Fall back to non-quantized model.. Fall back to non-quantized model.neon

Quantization is not supported for ArchType::. Fall back to non-quantized model.neonQuantization is not supported for ArchType::Quantization is not supported for ArchType::Quantization is not supported for ArchType::
neon. Fall back to non-quantized model.. Fall back to non-quantized model.

neonneon. Fall back to non-quantized model.. Fall back to non-quantized model.

neonQuantization is not supported for ArchType::neon. Fall back to non-quantized model.
neon. Fall back to non-quantized model.Quantization is not supported for ArchType::
. Fall back to non-quantized model.neon
. Fall back to non-quantized model.


[('직원 친절', 14700),
 ('가격 대비', 10120),
 ('사장 친절', 8028),
 ('방문 의사', 7897),
 ('가락 국수', 7196),
 ('평양 냉면', 6134),
 ('무한 리필', 3737),
 ('수제 버거', 3433),
 ('크림 치즈', 3052),
 ('치즈 케이크', 3035)]

In [12]:
for i in top_bigrams[:10]:
    print(i[0])

service = ["직원 친절", "사장 친절", "방문 의사"]
price = ["가격 대비", "무한 리필", "방문 의사"]
food = ["가락 국수", "수제 버거", "크림 치즈", "치즈 케이크"]

print({k: v for k, v in top_bigrams[:10]})


직원 친절
가격 대비
사장 친절
방문 의사
가락 국수
평양 냉면
무한 리필
수제 버거
크림 치즈
치즈 케이크
{'직원 친절': 14700, '가격 대비': 10120, '사장 친절': 8028, '방문 의사': 7897, '가락 국수': 7196, '평양 냉면': 6134, '무한 리필': 3737, '수제 버거': 3433, '크림 치즈': 3052, '치즈 케이크': 3035}


In [13]:
categories = {
    "service": set(service),
    "price": set(price),
    "food": set(food),
}

category_json = {k: {} for k in categories}

for phrase, count in top_bigrams:   # ← 슬라이스 제거 권장
    for cat, vocab in categories.items():
        if phrase in vocab:
            category_json[cat][phrase] = count
            
category_json

{'service': {'직원 친절': 14700, '사장 친절': 8028, '방문 의사': 7897},
 'price': {'가격 대비': 10120, '방문 의사': 7897, '무한 리필': 3737},
 'food': {'가락 국수': 7196, '수제 버거': 3433, '크림 치즈': 3052, '치즈 케이크': 3035}}

In [14]:
candidates = bigram_counts.takeOrdered(2000, key=lambda x: -x[1])

In [15]:
def is_noise(phrase):
    a, b = phrase.split()
    if len(a) < 2 or len(b) < 2:
        return True
    return False

candidates = [x for x in candidates if not is_noise(x[0])]

In [16]:
service_kw = ["친절", "서비스", "응대", "직원", "사장", "불친절"]
price_kw   = ["가격", "가성비", "대비", "리필", "무한", "할인", "쿠폰"]
food_kw    = ["국수", "냉면", "버거", "치즈", "케이크", "고기", "커피", "피자", "파스타"]

def classify(phrase):
    labels = []
    if any(k in phrase for k in service_kw): labels.append("service")
    if any(k in phrase for k in price_kw):   labels.append("price")
    if any(k in phrase for k in food_kw):    labels.append("food")
    if not labels: labels.append("other")
    return labels


In [17]:
category_json = {"service": {}, "price": {}, "food": {}, "other": {}}

for phrase, count in candidates:
    for lab in classify(phrase):
        category_json[lab][phrase] = count
category_json["service"]


{'직원 친절': 14700,
 '사장 친절': 8028,
 '친절 음식': 1932,
 '친절 기분': 1910,
 '서비스 친절': 1802,
 '친절 서비스': 1502,
 '분위기 서비스': 1236,
 '사장 직원': 1078,
 '친절 사장': 1042,
 '직원 서비스': 1038,
 '친절 직원': 1005,
 '음식 서비스': 983,
 '친절 방문': 979,
 '가격 서비스': 946,
 '친절 가격': 945,
 '친절 분위기': 940,
 '서비스 만족': 867,
 '서비스 분위기': 857,
 '분위기 친절': 805,
 '분위기 직원': 776,
 '친절 설명': 760,
 '친절 고기': 741,
 '종업원 친절': 735,
 '서비스 음식': 729,
 '아주머니 친절': 728,
 '음식 직원': 707,
 '서비스 최악': 655,
 '서버 친절': 617,
 '친절 가게': 608,
 '주인 친절': 589,
 '고기 직원': 572,
 '가격 친절': 570,
 '서비스 가격': 568,
 '서빙 친절': 558,
 '아르바이트 친절': 545,
 '음식 친절': 533,
 '이모 친절': 518,
 '남자 직원': 509,
 '셰프 친절': 508,
 '직원 교육': 508,
 '서비스 최고': 491,
 '친절 응대': 486,
 '친절 메뉴': 472,
 '친절 만족': 471,
 '사람 친절': 470,
 '사람 직원': 457,
 '서비스 기대': 452,
 '친절 커피': 452,
 '직원 고기': 419,
 '주문 직원': 411,
 '분위기 사장': 409,
 '남자 사장': 407,
 '사장 서비스': 405,
 '직원 사장': 397,
 '고기 서비스': 396,
 '친절 사람': 390,
 '친절 매장': 375,
 '서비스 엉망': 363,
 '친절 주문': 356,
 '가게 사장': 356,
 '추천 서비스': 352,
 '친절 최고': 350,
 '가격 직원': 350,
 '서비스 마인드': 34

In [18]:
category_json["price"]

{'가격 대비': 10120,
 '무한 리필': 3737,
 '가격 생각': 2993,
 '음식 가격': 1582,
 '합리 가격': 1412,
 '생각 가격': 1371,
 '분위기 가격': 1347,
 '메뉴 가격': 1340,
 '가격 만족': 1209,
 '가격 분위기': 1206,
 '가격 합리': 1204,
 '가격 부담': 1202,
 '가격 음식': 1168,
 '리필 가능': 1157,
 '커피 가격': 1010,
 '고기 가격': 952,
 '가격 서비스': 946,
 '친절 가격': 945,
 '가격 퀄리티': 937,
 '가격 구성비': 916,
 '가격 사악': 896,
 '가격 고기': 880,
 '음료 가격': 820,
 '주문 가격': 690,
 '가격 방문': 686,
 '대비 만족': 677,
 '가격 메뉴': 649,
 '부담 가격': 605,
 '방문 가격': 586,
 '가격 친절': 570,
 '반찬 리필': 569,
 '서비스 가격': 568,
 '대비 가격': 510,
 '파스타 가격': 488,
 '만족 가격': 481,
 '사람 가격': 472,
 '가격 커피': 463,
 '구성비 가격': 462,
 '식당 가격': 458,
 '추천 가격': 446,
 '퀄리티 가격': 440,
 '가격 사람': 417,
 '최고 가격': 412,
 '전체 가격': 409,
 '카페 가격': 405,
 '가격 추천': 403,
 '기대 가격': 400,
 '케이크 가격': 388,
 '가격 구성': 379,
 '가격 최고': 371,
 '세트 가격': 366,
 '가격 가게': 353,
 '가격 직원': 350,
 '요리 가격': 343,
 '비교 가격': 338,
 '가격 재료': 334,
 '식사 가격': 330,
 '버거 가격': 328,
 '가격 식사': 322,
 '구성 가격': 320,
 '대비 음식': 316,
 '가격 차이': 312,
 '가격 반찬': 307,
 '가격 기대': 306,
 '웨이팅 가격': 299

In [19]:
category_json["food"]

{'가락 국수': 7196,
 '평양 냉면': 6134,
 '수제 버거': 3433,
 '크림 치즈': 3052,
 '치즈 케이크': 3035,
 '크림 파스타': 2955,
 '당근 케이크': 1632,
 '오일 파스타': 1498,
 '카페 커피': 1494,
 '비빔 냉면': 1452,
 '치즈 버거': 1451,
 '리코타 치즈': 1418,
 '화덕 피자': 1404,
 '드립 커피': 1402,
 '피자 파스타': 1398,
 '로제 파스타': 1336,
 '국물 고기': 1333,
 '국수 국물': 1332,
 '분위기 커피': 1315,
 '고기 국물': 1309,
 '고기 국수': 1253,
 '고기 냄새': 1228,
 '피자 도우': 1226,
 '파스타 피자': 1168,
 '치즈 돈가스': 1140,
 '고기 만두': 1107,
 '커피 디저트': 1098,
 '커피 가격': 1010,
 '비빔 막국수': 1003,
 '고기 소스': 1001,
 '오리 고기': 972,
 '고기 가격': 952,
 '냉면 육수': 916,
 '칼국수 국물': 916,
 '소스 고기': 899,
 '크림 커피': 898,
 '가격 고기': 880,
 '커피 분위기': 874,
 '고기 육즙': 865,
 '치즈 샐러드': 838,
 '국수 사리': 830,
 '파운드 케이크': 809,
 '고기 육수': 797,
 '파스타 소스': 797,
 '파스타 스테이크': 792,
 '치즈 피자': 783,
 '커피 카페': 776,
 '고기 자체': 771,
 '고기 양념': 770,
 '아보카도 버거': 768,
 '잔치 국수': 758,
 '피자 치즈': 757,
 '생각 고기': 757,
 '커피 원두': 749,
 '비엔나 커피': 746,
 '크림 케이크': 742,
 '친절 고기': 741,
 '고기 튀김': 740,
 '커피 크림': 737,
 '고기 야채': 727,
 '딸기 케이크': 720,
 '커피 산미': 718,
 '만두 고기': 674,

In [20]:
category_json["other"]

{'방문 의사': 7897,
 '남자 친구': 2866,
 '새우 튀김': 2843,
 '종류 다양': 2607,
 '주차 공간': 2359,
 '주문 메뉴': 2235,
 '간장 게장': 2224,
 '여자 친구': 2185,
 '세트 메뉴': 2172,
 '메뉴 주문': 2153,
 '가게 분위기': 2149,
 '분위기 음식': 2126,
 '시그니처 메뉴': 2100,
 '카페 분위기': 1991,
 '대표 메뉴': 1949,
 '플랫 화이트': 1865,
 '메뉴 다양': 1845,
 '브레이크 타임': 1834,
 '테이블 간격': 1832,
 '만족 식사': 1791,
 '추가 주문': 1760,
 '사이드 메뉴': 1742,
 '취향 저격': 1724,
 '토마토 소스': 1700,
 '구성비 최고': 1691,
 '주문 가능': 1690,
 '코스 요리': 1689,
 '추천 메뉴': 1661,
 '방문 추천': 1652,
 '생선 구이': 1649,
 '마제 소바': 1640,
 '주차 가능': 1607,
 '가게 내부': 1527,
 '크림 소스': 1521,
 '망고 플레이트': 1516,
 '평일 저녁': 1503,
 '분위기 카페': 1490,
 '주문 음식': 1463,
 '사리 추가': 1463,
 '육회 비빔밥': 1450,
 '짬뽕 국물': 1435,
 '메인 메뉴': 1412,
 '순대 국밥': 1407,
 '기대 실망': 1351,
 '평일 점심': 1346,
 '메뉴 구성': 1309,
 '개인 취향': 1308,
 '양념 갈비': 1245,
 '저녁 방문': 1240,
 '내부 인테리어': 1238,
 '아이스 아메리카노': 1236,
 '방문 예정': 1226,
 '기대 생각': 1220,
 '분위기 방문': 1176,
 '국물 깔끔': 1175,
 '친구 추천': 1169,
 '방문 웨이팅': 1150,
 '에그 타르트': 1145,
 '음식 분위기': 1132,
 '창가 자리': 1116,
 '인테리어 분위기': 1

In [21]:
service_pairs = category_json["service"]
price_pairs = category_json["price"]
food_pairs = category_json["food"]
other_pairs = category_json["other"]

In [22]:
def quantile_split(pairs, head_q=0.02, mid_q=0.20, min_head=10):
    if not pairs:
        return [], [], []

    # ✅ dict이면 items로 변환
    if isinstance(pairs, dict):
        pairs = list(pairs.items())

    # ✅ 타입 검증
    if isinstance(pairs[0], str):
        raise ValueError("pairs must be [(phrase, count), ...] or dict{phrase:count}")

    pairs_sorted = sorted(pairs, key=lambda x: x[1], reverse=True)
    n = len(pairs_sorted)

    head_n = max(min_head, int(n * head_q))
    head_n = min(head_n, n)

    mid_end = int(n * mid_q)
    mid_end = max(mid_end, head_n)
    mid_end = min(mid_end, n)

    head = pairs_sorted[:head_n]
    mid  = pairs_sorted[head_n:mid_end]
    tail = pairs_sorted[mid_end:]
    return head, mid, tail

pairs_list = [service_pairs, price_pairs, food_pairs, other_pairs]
pairs_name = ["service", "price", "food", "other"]

for name, pairs in zip(pairs_name,pairs_list):
    head, mid, tail = quantile_split(pairs, head_q=0.02, mid_q=0.20, min_head=10)

    print(f"\n===== {name} =====")
    print(f"n={len(pairs)} | head={len(head)} | mid={len(mid)} | tail={len(tail)}")

    print("\n[HEAD]")
    print(head[:20])   # 너무 길면 일부만

    print("\n[MID]")
    print(mid[:20])

    print("\n[TAIL]")
    print(tail[:20])



===== service =====
n=142 | head=10 | mid=18 | tail=114

[HEAD]
[('직원 친절', 14700), ('사장 친절', 8028), ('친절 음식', 1932), ('친절 기분', 1910), ('서비스 친절', 1802), ('친절 서비스', 1502), ('분위기 서비스', 1236), ('사장 직원', 1078), ('친절 사장', 1042), ('직원 서비스', 1038)]

[MID]
[('친절 직원', 1005), ('음식 서비스', 983), ('친절 방문', 979), ('가격 서비스', 946), ('친절 가격', 945), ('친절 분위기', 940), ('서비스 만족', 867), ('서비스 분위기', 857), ('분위기 친절', 805), ('분위기 직원', 776), ('친절 설명', 760), ('친절 고기', 741), ('종업원 친절', 735), ('서비스 음식', 729), ('아주머니 친절', 728), ('음식 직원', 707), ('서비스 최악', 655), ('서버 친절', 617)]

[TAIL]
[('친절 가게', 608), ('주인 친절', 589), ('고기 직원', 572), ('가격 친절', 570), ('서비스 가격', 568), ('서빙 친절', 558), ('아르바이트 친절', 545), ('음식 친절', 533), ('이모 친절', 518), ('남자 직원', 509), ('셰프 친절', 508), ('직원 교육', 508), ('서비스 최고', 491), ('친절 응대', 486), ('친절 메뉴', 472), ('친절 만족', 471), ('사람 친절', 470), ('사람 직원', 457), ('서비스 기대', 452), ('친절 커피', 452)]

===== price =====
n=118 | head=10 | mid=13 | tail=95

[HEAD]
[('가격 대비', 10120), ('무한 리필', 3737), ('가격 생각', 2993)

In [23]:
import random, math

def pick_seeds(head, mid, tail, mid_k=5, tail_k=1, seed=42):
    random.seed(seed)

    # head는 전부 포함
    seeds = [p for p, _ in head]

    # mid는 가중 샘플링 (log(count))
    if mid:
        weights = [math.log(c + 1) for _, c in mid]
        idxs = list(range(len(mid)))
        for _ in range(min(mid_k, len(mid))):
            total = sum(weights[i] for i in idxs)
            r = random.random() * total
            acc = 0.0
            for i in idxs:
                acc += weights[i]
                if acc >= r:
                    seeds.append(mid[i][0])
                    idxs.remove(i)
                    break

    # tail은 랜덤 (탐색용)
    if tail and tail_k > 0:
        picks = random.sample(tail, k=min(tail_k, len(tail)))
        seeds.extend([p for p, _ in picks])

    # 중복 제거(순서 유지)
    seen = set()
    seeds = [s for s in seeds if not (s in seen or seen.add(s))]

    return seeds


In [24]:
seeds_list = []
for name, pairs in zip(pairs_name, pairs_list):
    head, mid, tail = quantile_split(pairs)
    seeds = pick_seeds(head, mid, tail, mid_k=5, tail_k=1)

    print(f"\n{name} seeds ({len(seeds)}):")
    print(seeds)
    seeds_list.append(seeds)


service seeds (16):
['직원 친절', '사장 친절', '친절 음식', '친절 기분', '서비스 친절', '친절 서비스', '분위기 서비스', '사장 직원', '친절 사장', '직원 서비스', '친절 고기', '친절 직원', '친절 분위기', '친절 가격', '아주머니 친절', '깔끔 친절']

price seeds (16):
['가격 대비', '무한 리필', '가격 생각', '음식 가격', '합리 가격', '생각 가격', '분위기 가격', '메뉴 가격', '가격 만족', '가격 분위기', '가격 퀄리티', '가격 합리', '리필 가능', '커피 가격', '가격 사악', '런치 가격']

food seeds (16):
['가락 국수', '평양 냉면', '수제 버거', '크림 치즈', '치즈 케이크', '크림 파스타', '당근 케이크', '오일 파스타', '카페 커피', '비빔 냉면', '커피 원두', '리코타 치즈', '비빔 막국수', '치즈 돈가스', '커피 산미', '치즈 파스타']

other seeds (33):
['방문 의사', '남자 친구', '새우 튀김', '종류 다양', '주차 공간', '주문 메뉴', '간장 게장', '여자 친구', '세트 메뉴', '메뉴 주문', '가게 분위기', '분위기 음식', '시그니처 메뉴', '카페 분위기', '대표 메뉴', '플랫 화이트', '메뉴 다양', '브레이크 타임', '테이블 간격', '만족 식사', '추가 주문', '사이드 메뉴', '취향 저격', '토마토 소스', '구성비 최고', '주문 가능', '코스 요리', '만족 방문', '가게 내부', '예약 필수', '김치 만두', '방문 만족', '카페 자리']


In [25]:
def dedup_reversed_bigrams(bigrams):
    seen = set()
    result = []

    for bg in bigrams:
        a, b = bg.split()
        key = tuple(sorted([a, b]))  # 순서 무시용 canonical key

        if key not in seen:
            seen.add(key)
            result.append(bg)

    return result


In [26]:
service_name_list = ["service","price","food","other"]

all_seeds = {
    "service": {},
    "price": {},
    "food": {},
    "other": {},
}

for seed, name in zip(seeds_list, service_name_list):
    cleaned = dedup_reversed_bigrams(seed)
    all_seeds[name] = cleaned

print(all_seeds["service"])
print(all_seeds["price"])
print(all_seeds["food"])
print(all_seeds["other"])


['직원 친절', '사장 친절', '친절 음식', '친절 기분', '서비스 친절', '분위기 서비스', '사장 직원', '직원 서비스', '친절 고기', '친절 분위기', '친절 가격', '아주머니 친절', '깔끔 친절']
['가격 대비', '무한 리필', '가격 생각', '음식 가격', '합리 가격', '분위기 가격', '메뉴 가격', '가격 만족', '가격 퀄리티', '리필 가능', '커피 가격', '가격 사악', '런치 가격']
['가락 국수', '평양 냉면', '수제 버거', '크림 치즈', '치즈 케이크', '크림 파스타', '당근 케이크', '오일 파스타', '카페 커피', '비빔 냉면', '커피 원두', '리코타 치즈', '비빔 막국수', '치즈 돈가스', '커피 산미', '치즈 파스타']
['방문 의사', '남자 친구', '새우 튀김', '종류 다양', '주차 공간', '주문 메뉴', '간장 게장', '여자 친구', '세트 메뉴', '가게 분위기', '분위기 음식', '시그니처 메뉴', '카페 분위기', '대표 메뉴', '플랫 화이트', '메뉴 다양', '브레이크 타임', '테이블 간격', '만족 식사', '추가 주문', '사이드 메뉴', '취향 저격', '토마토 소스', '구성비 최고', '주문 가능', '코스 요리', '만족 방문', '가게 내부', '예약 필수', '김치 만두', '카페 자리']


service_seeds = ['직원 친절', '사장 친절', '친절 기분', '서비스 친절', '사장 직원', '직원 서비스', '아주머니 친절']
price_seeds = ['가격 대비', '무한 리필', '가격 생각', '음식 가격', '합리 가격', '메뉴 가격', '가격 만족', '가격 퀄리티', '리필 가능', '커피 가격', '가격 사악', '런치 가격']
food_seeds = ['가락 국수', '평양 냉면', '수제 버거', '크림 치즈', '치즈 케이크', '크림 파스타', '당근 케이크', '오일 파스타', '카페 커피', '비빔 냉면', '커피 원두', '리코타 치즈', '비빔 막국수', '치즈 돈가스', '커피 산미', '치즈 파스타']




## (phrase, count) 버전 - 비중 구하기 (전체 음식점 리뷰 기준 kr3)

In [ ]:
service_pairs = category_json["service"]
price_pairs = category_json["price"]
food_pairs = category_json["food"]
other_pairs = category_json["other"]

In [27]:
def quantile_split(pairs, head_q=0.02, mid_q=0.20, min_head=10):
    if not pairs:
        return [], [], []

    # ✅ dict이면 items로 변환
    if isinstance(pairs, dict):
        pairs = list(pairs.items())

    # ✅ 타입 검증
    if isinstance(pairs[0], str):
        raise ValueError("pairs must be [(phrase, count), ...] or dict{phrase:count}")

    pairs_sorted = sorted(pairs, key=lambda x: x[1], reverse=True)
    n = len(pairs_sorted)

    head_n = max(min_head, int(n * head_q))
    head_n = min(head_n, n)

    mid_end = int(n * mid_q)
    mid_end = max(mid_end, head_n)
    mid_end = min(mid_end, n)

    head = pairs_sorted[:head_n]
    mid  = pairs_sorted[head_n:mid_end]
    tail = pairs_sorted[mid_end:]
    return head, mid, tail

pairs_list = [service_pairs, price_pairs, food_pairs, other_pairs]
pairs_name = ["service", "price", "food", "other"]

for name, pairs in zip(pairs_name,pairs_list):
    head, mid, tail = quantile_split(pairs, head_q=0.02, mid_q=0.20, min_head=10)

    print(f"\n===== {name} =====")
    print(f"n={len(pairs)} | head={len(head)} | mid={len(mid)} | tail={len(tail)}")

    print("\n[HEAD]")
    print(head[:20])   # 너무 길면 일부만

    print("\n[MID]")
    print(mid[:20])

    print("\n[TAIL]")
    print(tail[:20])


===== service =====
n=142 | head=10 | mid=18 | tail=114

[HEAD]
[('직원 친절', 14700), ('사장 친절', 8028), ('친절 음식', 1932), ('친절 기분', 1910), ('서비스 친절', 1802), ('친절 서비스', 1502), ('분위기 서비스', 1236), ('사장 직원', 1078), ('친절 사장', 1042), ('직원 서비스', 1038)]

[MID]
[('친절 직원', 1005), ('음식 서비스', 983), ('친절 방문', 979), ('가격 서비스', 946), ('친절 가격', 945), ('친절 분위기', 940), ('서비스 만족', 867), ('서비스 분위기', 857), ('분위기 친절', 805), ('분위기 직원', 776), ('친절 설명', 760), ('친절 고기', 741), ('종업원 친절', 735), ('서비스 음식', 729), ('아주머니 친절', 728), ('음식 직원', 707), ('서비스 최악', 655), ('서버 친절', 617)]

[TAIL]
[('친절 가게', 608), ('주인 친절', 589), ('고기 직원', 572), ('가격 친절', 570), ('서비스 가격', 568), ('서빙 친절', 558), ('아르바이트 친절', 545), ('음식 친절', 533), ('이모 친절', 518), ('남자 직원', 509), ('셰프 친절', 508), ('직원 교육', 508), ('서비스 최고', 491), ('친절 응대', 486), ('친절 메뉴', 472), ('친절 만족', 471), ('사람 친절', 470), ('사람 직원', 457), ('서비스 기대', 452), ('친절 커피', 452)]

===== price =====
n=118 | head=10 | mid=13 | tail=95

[HEAD]
[('가격 대비', 10120), ('무한 리필', 3737), ('가격 생각', 2993)

In [28]:
import random, math

def pick_seeds_pairs(head, mid, tail, mid_k=5, tail_k=1, seed=42):
    random.seed(seed)

    # ✅ head는 전부 포함 (p,c) 그대로
    seeds = list(head)

    # ✅ mid는 가중 샘플링 (log(count))
    if mid:
        idxs = list(range(len(mid)))
        weights = [math.log(c + 1) for _, c in mid]

        for _ in range(min(mid_k, len(mid))):
            total = sum(weights[i] for i in idxs)
            r = random.random() * total
            acc = 0.0
            for i in idxs:
                acc += weights[i]
                if acc >= r:
                    seeds.append(mid[i])   # ✅ (p,c) 그대로
                    idxs.remove(i)
                    break

    # ✅ tail은 랜덤 (탐색용)
    if tail and tail_k > 0:
        picks = random.sample(tail, k=min(tail_k, len(tail)))
        seeds.extend(picks)  # ✅ (p,c) 그대로

    # ✅ 중복 제거(phrase 기준, 순서 유지)
    seen = set()
    out = []
    for p, c in seeds:
        if p not in seen:
            out.append((p, c))
            seen.add(p)

    return out


In [29]:
def dedup_reversed_bigrams_pairs(pairs):
    # pairs: [( "a b", count ), ...]
    order = []         # canonical key의 등장 순서 유지
    agg = {}           # canonical key -> {"repr":원문표현, "count":합}

    for phrase, cnt in pairs:
        a, b = phrase.split()
        key = tuple(sorted([a, b]))  # 순서 무시용 canonical key

        if key not in agg:
            agg[key] = {"repr": phrase, "count": cnt}
            order.append(key)
        else:
            agg[key]["count"] += cnt  # ✅ 합산 (원하면 max로 변경 가능)

    return [(agg[k]["repr"], agg[k]["count"]) for k in order]


In [30]:
seeds_list = []
for name, pairs in zip(pairs_name, pairs_list):
    head, mid, tail = quantile_split(pairs)

    seeds = pick_seeds_pairs(head, mid, tail, mid_k=5, tail_k=1)  # ✅ 변경

    print(f"\n{name} seeds ({len(seeds)}):")
    print(seeds)
    seeds_list.append(seeds)

service_name_list = ["service","price","food","other"]

all_seeds = {}
for seed_pairs, name in zip(seeds_list, service_name_list):
    cleaned = dedup_reversed_bigrams_pairs(seed_pairs)  # ✅ 변경
    all_seeds[name] = cleaned

print(all_seeds["service"])
print(all_seeds["price"])
print(all_seeds["food"])
print(all_seeds["other"])



service seeds (16):
[('직원 친절', 14700), ('사장 친절', 8028), ('친절 음식', 1932), ('친절 기분', 1910), ('서비스 친절', 1802), ('친절 서비스', 1502), ('분위기 서비스', 1236), ('사장 직원', 1078), ('친절 사장', 1042), ('직원 서비스', 1038), ('친절 고기', 741), ('친절 직원', 1005), ('친절 분위기', 940), ('친절 가격', 945), ('아주머니 친절', 728), ('깔끔 친절', 231)]

price seeds (16):
[('가격 대비', 10120), ('무한 리필', 3737), ('가격 생각', 2993), ('음식 가격', 1582), ('합리 가격', 1412), ('생각 가격', 1371), ('분위기 가격', 1347), ('메뉴 가격', 1340), ('가격 만족', 1209), ('가격 분위기', 1206), ('가격 퀄리티', 937), ('가격 합리', 1204), ('리필 가능', 1157), ('커피 가격', 1010), ('가격 사악', 896), ('런치 가격', 212)]

food seeds (16):
[('가락 국수', 7196), ('평양 냉면', 6134), ('수제 버거', 3433), ('크림 치즈', 3052), ('치즈 케이크', 3035), ('크림 파스타', 2955), ('당근 케이크', 1632), ('오일 파스타', 1498), ('카페 커피', 1494), ('비빔 냉면', 1452), ('커피 원두', 749), ('리코타 치즈', 1418), ('비빔 막국수', 1003), ('치즈 돈가스', 1140), ('커피 산미', 718), ('치즈 파스타', 215)]

other seeds (33):
[('방문 의사', 7897), ('남자 친구', 2866), ('새우 튀김', 2843), ('종류 다양', 2607), ('주차 공간', 2359), ('주문 메뉴'

In [34]:
service_pairs = all_seeds["service"]
price_pairs = all_seeds["price"]
food_pairs = all_seeds["food"]
other_pairs = all_seeds["other"]

In [35]:
def sum_counts(pairs):
    return sum(cnt for _, cnt in pairs)

In [36]:
pairs_list = [service_pairs, price_pairs, food_pairs, other_pairs]
pairs_name = ["service", "price", "food", "other"]

total_by_category = {}

for name, pairs in zip(pairs_name, pairs_list):
    total = sum_counts(pairs)
    total_by_category[name] = total
    print(f"{name}: {total}")


service: 38858
price: 31733
food: 37124
other: 66216


In [ ]:
total_count = sum(total_by_category.values())
service_count = total_by_category["service"]
price_count = total_by_category["price"]
food_count = total_by_category["food"]
other_count = total_by_category["other"]
print("각 카테고리별 빈도: ",total_by_category)
print("총 빈도: ",total_count)

service_ratio = service_count / total_count
service_ratio = round(service_ratio, 2)
print("서비스 비중: ",service_ratio)
price_ratio = price_count / total_count
price_ratio = round(price_ratio, 2)
print("가격 비중: ",price_ratio)
food_ratio = food_count / total_count
food_ratio = round(food_ratio, 2)
print("음식 비중: ",food_ratio)
other_ratio = other_count / total_count
other_ratio = round(other_ratio, 2)
print("기타 비중: ",other_ratio)


각 카테고리별 빈도:  {'service': 38858, 'price': 31733, 'food': 37124, 'other': 66216}
총 빈도:  173931
서비스 비중:  0.22
가격 비중:  0.18
음식 비중:  0.21
기타 비중:  0.38
